In [1]:
from binary_param import linear, constant
import pandas as pd
import numpy as np
from R32_bmimBF4_PR import configuration
from pyomo.environ import (ConcreteModel,
                           SolverFactory,
                           units as pyunits)

import idaes.logger as idaeslog
import pyomo.contrib.parmest.parmest as parmest

import pytest

from idaes.core import FlowsheetBlock
from idaes.generic_models.unit_models import Flash
from idaes.generic_models.properties.core.generic.generic_property import (
        GenericParameterBlock)

from idaes.core.util.model_statistics import degrees_of_freedom

In [2]:
def PR_model(data):
    """
    Function which returns initilized model.

    Args:
        data: pandas DataFrame with data

    Returns:
        initialized model
    """
    eps = 0.1
    init_kappa_2_1A = -0.02
    init_kappa_1_2A = -0.007
    m = ConcreteModel()

    m.fs = FlowsheetBlock(default={"dynamic": False})

    m.fs.properties = GenericParameterBlock(default=configuration)

    m.fs.state_block = m.fs.properties.state_block_class(
        default={"parameters": m.fs.properties,
                 "defined_state": True})
    x = float(data['x_R32'])+eps
    m.fs.state_block.flow_mol.fix(1)
    m.fs.state_block.temperature.fix(float(data["temperature"]))
    m.fs.state_block.pressure.fix(float(data["pressure"]))
    m.fs.state_block.mole_frac_comp['bmimBF4'].fix(1-x)
    m.fs.state_block.mole_frac_comp['R32'].fix(x)

    # parameter - kappa_A_ij (set at 0.3, 0 if i=j)
    m.fs.properties.PR_kappa_A['bmimBF4','bmimBF4'].fix(0)
    m.fs.properties.PR_kappa_A['bmimBF4', 'R32'].fix(init_kappa_2_1A)
    m.fs.properties.PR_kappa_A['R32', 'R32'].fix(0)
    m.fs.properties.PR_kappa_A['R32', 'bmimBF4'].fix(init_kappa_1_2A)
    m.fs.properties.PR_kappa_B['bmimBF4', 'bmimBF4'].fix(0)
    m.fs.properties.PR_kappa_B['bmimBF4', 'R32'].fix(0)
    m.fs.properties.PR_kappa_B['R32', 'R32'].fix(0)
    m.fs.properties.PR_kappa_B['R32', 'bmimBF4'].fix(0)
    m.fs.properties.PR_kappa_C['bmimBF4', 'bmimBF4'].fix(0)
    m.fs.properties.PR_kappa_C['bmimBF4', 'R32'].fix(0)
    m.fs.properties.PR_kappa_C['R32', 'R32'].fix(0)
    m.fs.properties.PR_kappa_C['R32', 'bmimBF4'].fix(0)
    # Initialize the flash unit
    m.fs.state_block.initialize(outlvl=idaeslog.CRITICAL)

    # Fix the state variables on the state block
    m.fs.state_block.pressure.unfix()
    m.fs.state_block.mole_frac_comp['bmimBF4'].unfix()
    m.fs.state_block.mole_frac_comp['R32'].unfix()
    m.fs.state_block.temperature.fix(float(data["temperature"]))
    m.fs.state_block.mole_frac_phase_comp['Liq', 'R32'].fix(float(data['x_R32']))
    m.fs.state_block.mole_frac_phase_comp['Liq', 'bmimBF4'].fix(float(data['x_bmimBF4']))
    m.fs.state_block.mole_frac_comp['R32'].fix(float(data['x_R32'])+eps)
    m.fs.state_block.mole_frac_comp['bmimBF4'].unfix()

    # Set bounds on variables to be estimated
    m.fs.properties.PR_kappa_A['bmimBF4', 'R32'].setlb(-5)
    m.fs.properties.PR_kappa_A['bmimBF4', 'R32'].setub(5)

    m.fs.properties.PR_kappa_A['R32', 'bmimBF4'].setlb(-5)
    m.fs.properties.PR_kappa_A['R32', 'bmimBF4'].setub(5)

    # Return initialized flash model
    return m

def SSE(m, data):
    """
    returns objective function expresion.

    Args:
        m: model
        data: pandas DataFrame with data

    Returns:
        objectuve function scaled expresion
    """
    expr = ((float(data["pressure"]) - m.fs.state_block.pressure)**2)

    return expr * 1e-6

variable_name = ["fs.properties.PR_kappa_A['bmimBF4', 'R32']",
                 "fs.properties.PR_kappa_A['R32', 'bmimBF4']"]

In [3]:
data = pd.read_csv('R32_bmimBF4.csv')

In [4]:
N = 100 #number of samples
results_params = np.zeros((N,2))
results_obj = np.zeros((N,1))

st_dev_T = 0.01 #C
st_dev_P = 9771 #MPa
st_dev_x = 0.10 #mole fraction

for i in range(N):
    print(i)

    noise_x = np.random.normal(loc=0,scale=st_dev_x)
    print(noise_x)
    data_ = data.copy()

    data_["x_bmimBF4"] += data_["x_bmimBF4"] * noise_x

    data_.loc[data_['x_bmimBF4'] <= 0.0001, 'x_bmimBF4'] = 0.0001
    data_.loc[data_['x_bmimBF4'] >= 0.9999 , 'x_bmimBF4'] = 0.9999      

    try:
        pest = parmest.Estimator(PR_model, data_, variable_name, SSE, tee=False)

        obj_value, parameters = pest.theta_est()

        for k,v in parameters.items():
            print(k, "=", v)

        line = 0

        for k,v in parameters.items():
            results_params[i,line] = v
            line += 1
        results_obj[i] = obj_value

    except ValueError:
        results_params[i,0] = 'NaN'
        results_params[i,1] = 'NaN'


0
-0.020465711929663254
2022-03-06 00:22:12 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 00:22:13 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 00:22:14 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 00:22:15 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 00:22:16 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 00:22:17 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 00:22:18 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 00:22:18 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-

2022-03-06 00:23:54 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 00:23:55 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 00:23:56 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 00:23:58 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 00:23:59 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 00:24:00 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 00:24:01 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 00:24:02 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 00:24:03 [INFO] idaes

2022-03-06 00:25:22 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 00:25:23 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 00:25:24 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 00:25:25 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 00:25:27 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 00:25:28 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 00:25:29 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 00:25:30 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 00:25:31 [INFO] idaes

2022-03-06 00:26:49 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 00:26:50 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 00:26:51 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 00:26:52 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 00:26:53 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 00:26:54 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 00:26:55 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 00:26:56 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 00:26:58 [INFO] idaes

2022-03-06 00:28:23 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 00:28:24 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 00:28:26 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 00:28:27 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 00:28:28 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 00:28:29 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 00:28:31 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 00:28:32 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 00:28:33 [INFO] idaes

...

2022-03-06 01:14:07 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 01:14:08 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 01:14:09 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 01:14:10 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 01:14:10 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 01:14:11 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 01:14:12 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 01:14:13 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 01:14:13 [INFO] idaes

2022-03-06 01:15:07 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 01:15:08 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 01:15:09 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 01:15:10 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 01:15:10 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 01:15:11 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 01:15:12 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 01:15:13 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 01:15:13 [INFO] idaes

2022-03-06 01:16:07 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 01:16:08 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 01:16:08 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 01:16:09 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 01:16:10 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 01:16:11 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 01:16:12 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 01:16:12 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 01:16:13 [INFO] idaes

2022-03-06 01:17:06 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 01:17:07 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 01:17:08 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 01:17:08 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 01:17:09 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 01:17:10 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 01:17:11 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 01:17:11 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 01:17:12 [INFO] idaes

2022-03-06 01:18:06 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 01:18:06 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 01:18:07 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 01:18:08 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 01:18:09 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 01:18:09 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 01:18:10 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 01:18:11 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 01:18:12 [INFO] idaes

In [5]:
print(results_params)

[[-0.0362484  -0.00913176]
 [-0.04340605 -0.00957952]
 [-0.04139312 -0.00877706]
 [-0.0532403  -0.01424722]
 [-0.05092178 -0.01352756]
 [-0.06184717 -0.01698368]
 [-0.04088458 -0.00862782]
 [-0.04595705 -0.01201024]
 [-0.04345538 -0.01125756]
 [-0.07209816 -0.02038611]
 [-0.05417532 -0.01453949]
 [-0.04229862 -0.00907404]
 [-0.0428228  -0.00925061]
 [-0.05239433 -0.0139838 ]
 [-0.04009913 -0.00840396]
 [-0.04723095 -0.01239654]
 [-0.04139702 -0.01064406]
 [-0.03997032 -0.01022185]
 [-0.03977934 -0.00831674]
 [-0.02895051 -0.00704031]
 [-0.05252921 -0.01402573]
 [-0.04275428 -0.00922417]
 [-0.03722091 -0.00781893]
 [-0.04759647 -0.01250776]
 [-0.04238063 -0.00910477]
 [-0.04775213 -0.01255518]
 [-0.05967154 -0.01628202]
 [-0.04321662 -0.00943043]
 [-0.03110169 -0.00765063]
 [-0.03960764 -0.00827196]
 [-0.04017829 -0.00842622]
 [-0.04304009 -0.00984474]
 [-0.06449579 -0.0178473 ]
 [-0.03882298 -0.00807387]
 [-0.03924303 -0.00817693]
 [-0.0452844  -0.01180709]
 [-0.04510355 -0.01175257]
 

In [6]:
results = pd.DataFrame(results_params)
results.to_csv('R32_bmimbf4_10_2.csv')